# **BHASHA-AI**

## AI-Powered Multilingual Voice Assistant
### Translate between Hindi, English, Marathi, German, French & Spanish
### 🔊 Text & Voice Input/Output | 🤖 AI-Powered Translations

## FEATURES:
### - Beautiful Interactive Interface
### -6+ Language Support  
### -Text-to-Speech Output
### - Real-time Translation
### -Conversation History
### - Works in Google Colab & Hugging Face

## USAGE:
### 1. Run all cells sequentially
### 2. Use the web interface for translations
### 3. Choose input text & target language
### 4. Get instant translations with voice output

### Built with: Gradio, Googletrans, gTTS


## Install Required Libraries

In [2]:
# 🛠 Fix NumPy binary compatibility issue
!pip install --upgrade --force-reinstall numpy==1.26.4
!pip install --force-reinstall gradio googletrans==4.0.0-rc1 gtts

print("✅ Compatibility fixed! Please RESTART your runtime (Ctrl+M then restart) before running next cells.")


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.19.0 requires grpcio>=1.48.2, which is not installed.
tensorflow 2.19.0 requires grpcio<2.0,>=1.24.3, which is not installed.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 25.0 which is incompatible.
streamlit 1.32.0 requires rich<14,>=10.14.0, but you have rich 14.2.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Using cached gradio-6.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached gTTS-2.5.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached hstspreload-2025.1.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached hpa

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [3]:
import numpy as np
from googletrans import Translator
from gtts import gTTS
import gradio as gr
import tempfile
import time
import os

print("Libraries imported successfully!")

Libraries imported successfully!


## Core Translation and Speech Classes

In [8]:
class SimpleTranslator:
    def __init__(self):
        self.translator = Translator()

    def translate_text(self, text, source_lang, target_lang):
        try:
            if source_lang == 'auto':
                source_code = None
            else:
                lang_map = {'english': 'en', 'hindi': 'hi', 'marathi': 'mr', 'german': 'de'}
                source_code = lang_map[source_lang]

            lang_map = {'english': 'en', 'hindi': 'hi', 'marathi': 'mr', 'german': 'de'}
            target_code = lang_map[target_lang]

            translated = self.translator.translate(text, src=source_code, dest=target_code)
            return {'success': True, 'translated_text': translated.text}
        except Exception as e:
            return {'success': False, 'error': str(e)}

# Simple TTS Class
class SimpleTTS:
    def text_to_speech(self, text, language='english'):
        try:
            lang_map = {'english': 'en', 'hindi': 'hi', 'marathi': 'mr', 'german': 'de'}
            lang_code = lang_map[language]

            tts = gTTS(text=text, lang=lang_code, slow=False)
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
                tts.save(tmp_file.name)
                return tmp_file.name, "Audio ready"
        except Exception as e:
            return None, f"Error: {e}"

# Simple Assistant
class SimpleAssistant:
    def __init__(self):
        self.translator = SimpleTranslator()
        self.tts = SimpleTTS()
        self.history = []

    def process_translation(self, input_text, input_lang, output_lang, output_type):
        try:
            if not input_text.strip():
                return "Please enter text", None, []

            result = self.translator.translate_text(input_text, input_lang, output_lang)

            if not result['success']:
                return result['error'], None, []

            translated = result['translated_text']

            # Update history
            self.history.append({
                'input': input_text,
                'output': translated
            })
            if len(self.history) > 5:
                self.history.pop(0)

            output = f"**Input ({input_lang}):** {input_text}\n\n**Output ({output_lang}):** {translated}"

            audio = None
            if output_type == 'voice':
                audio, _ = self.tts.text_to_speech(translated, output_lang)

            return output, audio, self.history

        except Exception as e:
            return f"Error: {str(e)}", None, []

# Create assistant
assistant = SimpleAssistant()


## Premium 3D Gradio Interface (UI Layer)

In [10]:
pip install httpx==0.23.1


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.23.1 which is incompatible.



     ---------------------------------------- 0.0/52.5 kB ? eta -:--:--
     -------------- ----------------------- 20.5/52.5 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 52.5/52.5 kB 541.8 kB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/85.0 kB ? eta -:--:--
   ---------------------------------------- 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/69.6 kB ? eta -:--:--
   ---------------------------------------- 69.6/69.6 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
   ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfu

In [17]:
!pip install --upgrade gradio googletrans==4.0.0-rc1 gtts


Defaulting to user installation because normal site-packages is not writeable
  Using cached gradio-6.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached brotli-1.2.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached fastapi-0.121.3-py3-none-any.whl.metadata (30 kB)
  Using cached ffmpy-1.0.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-2.0.0.dev3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
  Using cached gradio-5.50.0-py3-none-any.whl.metadata (16 k

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires rich<14,>=10.14.0, but you have rich 14.2.0 which is incompatible.


In [18]:
import gradio as gr
import random
from googletrans import Translator
from gtts import gTTS
import tempfile
import time
import requests

# Accurate translation function
class WorkingTranslator:
    def __init__(self):
        self.history = []
        self.translator = Translator()

    def process_translation(self, input_text, target_lang, output_type):
        try:
            if not input_text.strip():
                return "Please enter text", None, self.history

            print(f"Translating: '{input_text}' to {target_lang}")  # Debug

            # Language mapping for googletrans
            lang_map = {
                'hindi': 'hi',
                'english': 'en',
                'marathi': 'mr',
                'german': 'de',
                'french': 'fr',
                'spanish': 'es'
            }

            target_code = lang_map.get(target_lang, 'en')

            # ALWAYS use googletrans for accurate translation
            translated = self.translator.translate(input_text, dest=target_code)
            translated_text = translated.text
            print(f"✅ Translation successful: {translated_text}")

            # Update history
            self.history.append({
                'input': input_text,
                'output': translated_text,
                'time': time.strftime("%H:%M:%S")
            })

            if len(self.history) > 5:
                self.history.pop(0)

            output = f"**Input:** {input_text}\n\n**Translated ({target_lang}):** {translated_text}"

            # Generate audio if needed
            audio_file = None
            if output_type == 'voice':
                try:
                    # Use the target language for TTS
                    tts_lang = target_code if target_code in ['en', 'hi', 'mr', 'de', 'fr', 'es'] else 'en'
                    tts = gTTS(translated_text, lang=tts_lang)
                    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
                        tts.save(f.name)
                        audio_file = f.name
                    print("✅ Audio file generated successfully")
                except Exception as e:
                    print(f"❌ TTS Error: {e}")
                    output += f"\n\n⚠️ Voice output failed: {e}"

            return output, audio_file, self.history

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"
            print(f"Translation error: {error_msg}")
            return error_msg, None, self.history

# Create working assistant
assistant = WorkingTranslator()

def create_premium_interface():
    def process_translation(input_text, target_lang, output_type):
        return assistant.process_translation(input_text, target_lang, output_type)

    # Custom CSS for 3D effects and premium design
    custom_css = """
       # Custom CSS - Simplified for dropdown functionality
    @import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');

    .gradio-container {
        font-family: 'Poppins', sans-serif !important;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        min-height: 100vh !important;
    }

    .main-container {
        background: rgba(255, 255, 255, 0.95) !important;
        backdrop-filter: blur(20px) !important;
        border-radius: 25px !important;
        border: 1px solid rgba(255, 255, 255, 0.2) !important;
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1) !important;
        margin: 20px !important;
        padding: 30px !important;
    }

    .logo-container {
        text-align: center;
        margin-bottom: 30px;
    }

    .logo {
        font-size: 4em;
        background: linear-gradient(45deg, #FF6B6B, #4ECDC4, #45B7D1, #96CEB4);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        filter: drop-shadow(0 5px 15px rgba(0,0,0,0.2));
        animation: float 3s ease-in-out infinite;
    }

    @keyframes float {
        0%, 100% { transform: translateY(0px); }
        50% { transform: translateY(-10px); }
    }

    .welcome-title {
        font-size: 3.5em !important;
        font-weight: 700 !important;
        background: linear-gradient(45deg, #FF6B6B, #4ECDC4) !important;
        -webkit-background-clip: text !important;
        -webkit-text-fill-color: transparent !important;
        background-clip: text !important;
        margin-bottom: 10px !important;
    }

    .welcome-subtitle {
        font-size: 1.3em !important;
        color: #666 !important;
        margin-bottom: 30px !important;
    }

    .glow-button {
        background: linear-gradient(45deg, #FF6B6B, #4ECDC4) !important;
        border: none !important;
        border-radius: 50px !important;
        padding: 20px 40px !important;
        font-size: 1.3em !important;
        font-weight: 600 !important;
        color: white !important;
        box-shadow: 0 10px 30px rgba(255, 107, 107, 0.3) !important;
        transition: all 0.3s ease !important;
        position: relative !important;
        overflow: hidden !important;
    }

    .glow-button:hover {
        transform: translateY(-3px) !important;
        box-shadow: 0 15px 40px rgba(255, 107, 107, 0.4) !important;
    }

    .input-box {
        border-radius: 20px !important;
        border: 2px solid #e0e0e0 !important;
        padding: 20px !important;
        font-size: 16px !important;
        transition: all 0.3s ease !important;
        background: rgba(255, 255, 255, 0.8) !important;
        width: 100% !important;
    }

    .input-box:focus {
        border-color: #4ECDC4 !important;
        box-shadow: 0 0 20px rgba(78, 205, 196, 0.2) !important;
    }

    /* Remove all custom dropdown styling - let Gradio handle it */
    .radio-group {
        background: rgba(255, 255, 255, 0.8) !important;
        border-radius: 15px !important;
        padding: 15px !important;
        border: 2px solid #e0e0e0 !important;
    }

    .output-box {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%) !important;
        border-radius: 20px !important;
        padding: 25px !important;
        border: none !important;
        min-height: 200px !important;
    }
    .output-box * {
        color: #333333 !important;
    }

    .particle {
        position: absolute;
        width: 4px;
        height: 4px;
        background: #4ECDC4;
        border-radius: 50%;
        animation: float 3s ease-in-out infinite;
        opacity: 0.7;
    }

    .settings-title {
        font-size: 1.2em !important;
        font-weight: 600 !important;
        color: #555 !important;
        margin-bottom: 15px !important;
        text-align: center !important;
    }
    """

    # Create particles for background
    particles_html = ""
    for i in range(20):
        top = random.randint(0, 100)
        left = random.randint(0, 100)
        delay = random.uniform(0, 5)
        particles_html += f'<div class="particle" style="top:{top}%; left:{left}%; animation-delay:{delay}s;"></div>'

    with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as interface:

        # Background particles
        gr.HTML(f"<div style='position: fixed; top: 0; left: 0; width: 100%; height: 100%; z-index: -1;'>{particles_html}</div>")

        # Main container
        with gr.Column(elem_classes="main-container"):

            # Welcome Screen
            with gr.Column(visible=True) as welcome_screen:
                gr.HTML("""
                <div class="logo-container">
                    <div class="logo">💬✨</div>
                </div>
                """)

                gr.HTML("""
                <div style="text-align: center;">
                    <div class="welcome-title">BhashaAI</div>
                    <div class="welcome-subtitle">Your Intelligent Multilingual Assistant</div>
                </div>
                """)

                start_btn = gr.Button("🚀 Start Translating", elem_classes="glow-button")

                gr.HTML("""
                <div style="text-align: center; margin-top: 30px;">
                    <div style="display: inline-block; background: rgba(78, 205, 196, 0.1); padding: 15px 30px; border-radius: 20px; border: 2px dashed #4ECDC4;">
                        <span style="color: #666; font-size: 14px;">✨ Supports 5+ Languages • 🎯 Text & Voice Output • 🤖 AI-Powered</span>
                    </div>
                </div>
                """)

            # Main Translation Screen
            with gr.Column(visible=False) as main_screen:
                # Header
                with gr.Row():
                    gr.HTML("""
                    <div style="text-align: center; width: 100%;">
                        <div class="logo" style="font-size: 2.5em; display: inline-block;">💬✨</div>
                        <h1 style="display: inline-block; margin-left: 15px; background: linear-gradient(45deg, #FF6B6B, #4ECDC4); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text;">BhashaAI</h1>
                    </div>
                    """)

                gr.HTML("""<div style="text-align: center; color: #666; font-size: 1.2em; margin-bottom: 30px;">Hi! I'm ready to translate for you 🌟</div>""")

                # Input Section
                with gr.Row():
                    with gr.Column(scale=1):
                        input_text = gr.Textbox(
                            label="",
                            placeholder="🎤 Type: hello, thank you, how are you, good morning",
                            lines=4,
                            elem_classes="input-box"
                        )

                        # Settings Section with improved layout
                        gr.HTML("""<div class="settings-title">⚙️ Translation Settings</div>""")
                        with gr.Row(elem_classes="settings-row"):
                            with gr.Column(elem_classes="settings-column"):
                                target_lang = gr.Dropdown(
                                    choices=["hindi", "english", "marathi", "german", "french", "spanish"],
                                    label="🎯 Target Language",
                                    value="hindi",
                                    info="Select the language you want to translate to",
                                    elem_classes="dropdown"
                                )

                            with gr.Column(elem_classes="settings-column"):
                                output_type = gr.Radio(
                                    choices=["text", "voice"],
                                    label="📢 Output Type",
                                    value="text",
                                    info="Choose how you want to receive the translation",
                                    elem_classes="radio-group"
                                )

                        translate_btn = gr.Button(
                            "🎯 Translate Magic",
                            size="lg",
                            elem_classes="glow-button"
                        )

                    with gr.Column(scale=1):
                        output_text = gr.Markdown(
                            label="",
                            value="**🌟 Your translation will appear here...**",
                            elem_classes="output-box"
                        )

                        audio_output = gr.Audio(
                            label="🔊 Voice Output",
                            visible=False
                        )

                # History Section
                with gr.Accordion("📜 Translation History", open=False):
                    history = gr.JSON(label="", show_label=False)

        # Navigation
        def show_main_screen():
            return {
                welcome_screen: gr.update(visible=False),
                main_screen: gr.update(visible=True)
            }

        # Audio visibility
        def update_audio_visibility(output_type):
            return gr.Audio(visible=(output_type == 'voice'))

        # Event handlers
        start_btn.click(show_main_screen, outputs=[welcome_screen, main_screen])

        output_type.change(
            update_audio_visibility,
            inputs=output_type,
            outputs=audio_output
        )

        translate_btn.click(
            process_translation,
            inputs=[input_text, target_lang, output_type],
            outputs=[output_text, audio_output, history]
        )

    return interface

print("🎨 Creating Premium Interface...")
print("🚀 Testing translation...")

# Test the translation function
test_result = assistant.process_translation("hello", "hindi", "text")
print("Test result:", test_result[0])

premium_interface = create_premium_interface()
print("✅ Interface ready! Launching...")
premium_interface.launch(share=True, debug=True)

🎨 Creating Premium Interface...
🚀 Testing translation...
Translating: 'hello' to hindi
✅ Translation successful: नमस्ते
Test result: **Input:** hello

**Translated (hindi):** नमस्ते
✅ Interface ready! Launching...
Running on local URL:  http://127.0.0.1:7867


Exception in thread Thread-38 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\gradio\analytics.py", line 63, in _do_normal_analytics_request
    httpx.post(url, data=data, timeout=5)
  File "C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\httpx\_api.py", line 269, in post
    return request(
           ^^^^^^^^
  File "C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\httpx\_api.py", line 86, in request
    return client.request(
           ^^^^^^^^^^^^^^^
  File "C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\httpx\_client.py", line 600, in request
    return self.send(
           ^^^^^^^^^^
  File "C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\httpx\_client.py", line 620, in send
    response = self.send_handling_redirects(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nehaa\AppData\Roaming\Python\Python312\site-packages\htt

AttributeError: module 'httpx' has no attribute 'ConnectError'